### How can data help us heal communities at high risk for suicide?

A person or vehicle is hit by a train about once every three hours.

This results in approximately 700 deaths per year, by accident and by suicide.

By creating descriptive models that examine empiric data on train fatalities and predictive models that can anticipate accidents and suicide attempts, we can decrease the number of deaths that occur and focus on communities that are at disproportinately high risk. Good data can be a focus national efforts to heal areas impacted by suicide, and to promote smart planning and routing to prevent accidents.

Data link: https://drive.google.com/file/d/0B-velHZJGPpOVF9ueHJsNWI0ZUk/view

**Useful Links:**

http://transweb.sjsu.edu/PDFs/research/1129-2-preventing-suicide-on-US-rail-systems.pdf

http://railwaysuicideprevention.com/railway-fatalities/overview.html#suicide

http://safetydata.fra.dot.gov/OfficeofSafety/default.aspx

In [93]:
# coding: utf-8
import csv as csv
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import MultiLabelBinarizer

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
matplotlib.style.use('ggplot')
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.cross_validation import ShuffleSplit
%matplotlib inline

import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline

In [94]:
DATA_FILE ='bayeshackgit/data/transportation-railroad-casualties.csv'
DATA_CODEMAPPED_FILE='bayeshackgit/data/transportation-railroad-casualties_Codemapped.csv'

In [3]:
def load(filepath):
    df = pd.read_csv(filepath, header=0, parse_dates=True)
    return df

In [4]:
data_df = load(DATA_FILE)
data_df_2011 = data_df[data_df['YEAR'] > 2010]
data_map_df = load(DATA_CODEMAPPED_FILE)
data_map_df_2011 = data_map_df[data_map_df['YEAR'] > 2010]

/opt/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:3006: DtypeWarning: Columns (9,10,11,20,22,23,24,25,26,28,30,31,32,33,34,37) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/opt/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:3006: DtypeWarning: Columns (10,21,23,26,27,29,31,38) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [111]:
import pymongo as pm
try:
    conn = pm.MongoClient('198.11.197.250', 27017)
    bayes_db = conn['db_bayes']
except pm.errors.ConnectionFailure, e:
    print "MongoDB Connection Failure:", e

In [6]:
def load_dataframe_into_mongo(df, col_mongo):
    for index, row in df.iterrows():
        data = {}
        for key in df.keys():
            data[key] = row[key]
        col_mongo.insert(data)

In [7]:
col_all_casualties = bayes_db['rail_casualties_2011']
load_dataframe_into_mongo(data_df_2011, col_all_casualties)
N = len(data_df_2011)

/opt/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


In [16]:
col_map_casualties_2011 = bayes_db['railmap_casualties_2011']
load_dataframe_into_mongo(data_df_2011, col_map_casualties_2011)


/opt/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


In [7]:
N = len(data_df_2011)

In [8]:
# Helpers (currently cut and paste around different notebooks... oops)

def percentify_axis(ax, which):
    which = which.lower()
    if which in ('x', 'both'):
        ax.set_xticklabels(['%.0f%%' % (t*100) for t in ax.get_xticks()])
    if which in ('y', 'both'):
        ax.set_yticklabels(['%.0f%%' % (t*100) for t in ax.get_yticks()])

color_idx = 0
CYCLE_COLORS = sns.color_palette()
def next_color():
    global color_idx
    c = CYCLE_COLORS[color_idx] 
    color_idx = (color_idx + 1) % len(CYCLE_COLORS)
    return c

def count_unique(s):
    values = s.unique()
    return sum(1 for v in values if pd.notnull(v))

def missing_pct(s):
    missing = N - s.count()
    return missing * 100.0 / N

def complete_pct(s):
    return 100 - missing_pct(s)

def summarize_completeness_uniqueness(df):
    print '*** How complete is each feature? How many different values does it have? ***'
    rows = []
    for col in df.columns:
        rows.append([col, '%.0f%%' % complete_pct(df[col]), count_unique(df[col])])
    return pd.DataFrame(rows, columns=['Column Name', 'Complete (%)','Unique Values'])

def summarize_completeness_over_time(df, time_col, transpose=True):
    print '*** Data completeness over time per column ***'
    x = df.groupby(time_col).count()
    x = x.div(df.groupby(time_col).size(), axis=0)
    for col in x.columns:
        x[col] = x[col].apply(lambda value: '%.0f%%' % (value * 100))
    if transpose:
        return x.T
    return x

def plot_top_hist(df, col, top_n=10, skip_below=.01):
    '''Plot a histogram of a categorical dataframe column, limiting to the most popular.'''
    counts = df[col].value_counts(True, ascending=True)
    missing = missing_pct(df[col])
    if counts.max() < (skip_below / (1 - missing)):
        print 'Skipping "%s" histogram -- most common value is < %.0f%% of all cases' % (col, skip_below*100)
        return
    fig, ax = plt.subplots(1)
    explanation = ''
    if len(counts) > top_n:
        explanation = ' (top %d of %d)' % (top_n, len(counts))
        counts = counts.iloc[-top_n:]
    explanation += ' -- %.0f%% missing' % (missing)
    counts.plot(kind='barh', ax=ax, color=next_color())
    ax.set_title('Rows by "%s"%s' % (col, explanation))
    ax.set_xticklabels(['%.0f%%' % (t*100) for t in ax.get_xticks()])

In [9]:
summarize_completeness_uniqueness(data_df_2011)

*** How complete is each feature? How many different values does it have? ***


,Column Name,Complete (%),Unique Values
0,YEAR,100%,5
1,MONTH,100%,12
2,DAY,100%,31
3,TIME24HR,100%,24
4,TIMEMIN,100%,60
5,SUICIDE_ATTEMPTED,100%,2
6,RAILROAD,100%,486
7,INCIDENT_NUM,100%,42981
8,TYPPERS,100%,10
9,JOBCODE,48%,115


In [10]:
def get_state_label(state_code):
    return numericcodes.dictSTATE[state_code]['desc']

def get_state_code(state_label):
    for item in numericcodes.dictSTATE:
        state = numericcodes.dictSTATE[item]['desc'].strip()
        if state_label == state:
            return item

In [11]:
data_map_df_2011[:10]

,Unnamed: 0,YEAR,MONTH,DAY,TIME24HR,TIMEMIN,SUICIDE_ATTEMPTED,RAILROAD,INCIDENT_NUM,TYPPERS,...,LOCC,EVENT,TOOLS,INJCAUS,HZMEXPOS,TERMINAT,COVERDATA,LATITUDE,LONGITUDE,NARRATIVE
1018220,1018220,2011,7,1,10,45,0,ARR,20110072,0,...,2,51,28,6,NaN,NaN,NaN,46.546999,-122.932443,TWISTED LEFT KNEE WALKING
1018221,1018221,2011,7,2,23,30,0,ARR,20110074,0,...,31,99,18,18,NaN,NaN,NaN,46.546999,-122.932443,EMPLOYEE FAINTED: STOOD UP TOO FAST AFTER CLEA...
1018222,1018222,2011,7,12,1,50,0,ARR,20110075,2,...,15,59,76,7,NaN,NaN,NaN,64.853163,-147.696041,RAN OVER TRESPASSER AT G-1.6 EIELSON BRANCH.
1018223,1018223,2011,7,13,19,10,0,ARR,20110077,2,...,4,70,13,17,NaN,NaN,NaN,61.218403,-149.906800,TRESPASSER ATTEMPT TO BOARD MOVING TRAIN AND W...
1018224,1018224,2011,7,30,16,0,0,ARR,20110085,0,...,25,44,19,18,NaN,NaN,NaN,61.218403,-149.906800,"STIFF NECK, PINCHED NERVE IN NECK."
1018225,1018225,2011,7,27,12,0,0,ARR,20110088,0,...,3,38,24,9,NaN,NaN,NaN,61.218403,-149.906800,BACK STRAIN WHILE WORKING WITH A CLAW BAR PULL...
1018226,1018226,2011,3,8,9,55,0,PARN,E110301,0,...,31,73,88,3,NaN,NaN,NaN,61.218403,-149.906800,EMPLOYEE WAS SERVICING SHOP HEAT PLANT BOILER ...
1018227,1018227,2011,3,2,13,30,0,ARR,20110014,0,...,31,52,88,2,NaN,NaN,NaN,61.218403,-149.906800,SLIPPED OFF BOTH FEET WHILE GETTING OUT OF THE...
1018228,1018228,2011,3,5,5,0,0,ARR,20110015,0,...,14,20,09,4,NaN,NaN,NaN,61.218403,-149.906800,EMPLOYEE FRACTURED AND LACERATED RIGHT MIDDLE ...
1018229,1018229,2011,11,2,13,30,0,ARR,20110130,0,...,1,99,22,9,NaN,NaN,NaN,61.218403,-149.906800,INJURED RIGHT ARM AND/OR SHOULDER REMOVING FIL...


In [40]:
poverty_income_file_2011 ='data/poverty_median_income_2011.csv'
data_pov_df_2011 = load(poverty_income_file_2011)
poverty_income_file_2012 ='data/poverty_median_income_2012.csv'
data_pov_df_2012 = load(poverty_income_file_2012)
poverty_income_file_2013 ='data/poverty_median_income_2013.csv'
data_pov_df_2013 = load(poverty_income_file_2013)
poverty_income_file_2014 ='data/poverty_median_income_2014.csv'
data_pov_df_2014 = load(poverty_income_file_2014)

In [41]:
data_pov_df_2014[:2]

,STATE,CNTYCD,STATEDESC,COUNTY,YEAR,POVERTY,POVERTYRATE,INCOME
0,0,0,US,United States,2014,"48,208,387",21.7,"53,657"
1,1,0,AL,Alabama,2014,"905,682",27.4,"42,917"


In [42]:
col_poverty_income_2011 = bayes_db['poverty_income_2011']
load_dataframe_into_mongo(data_pov_df_2011, col_poverty_income_2011)
load_dataframe_into_mongo(data_pov_df_2012, col_poverty_income_2011)
load_dataframe_into_mongo(data_pov_df_2013, col_poverty_income_2011)
load_dataframe_into_mongo(data_pov_df_2014, col_poverty_income_2011)


/opt/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


In [43]:
unemployment_file_2011 ='data/unemployemnt_county_2011.csv'
data_emp_df_2011 = load(unemployment_file_2011)
unemployment_file_2012 ='data/unemployemnt_county_2012.csv'
data_emp_df_2012 = load(unemployment_file_2012)
unemployment_file_2013 ='data/unemployemnt_county_2013.csv'
data_emp_df_2013 = load(unemployment_file_2013)
unemployment_file_2014 ='data/unemployemnt_county_2014.csv'
data_emp_df_2014 = load(unemployment_file_2014)

In [44]:
data_emp_df_2014[:2]

,STATE,CNTYCD,COUNTYNAME,COUNTY,STATEDESC,YEAR,FORCE,EMPLOYED,UNEMPLOYED,UNEMPLOYMENTRATE
0,1,1,"Autauga County, AL",Autauga County,AL,2014,25597,24097,1500,5.9
1,1,3,"Baldwin County, AL",Baldwin County,AL,2014,86400,81084,5316,6.2


In [45]:
col_unemployment_2011 = bayes_db['unemployment_2011']
load_dataframe_into_mongo(data_emp_df_2011, col_unemployment_2011)
load_dataframe_into_mongo(data_emp_df_2012, col_unemployment_2011)
load_dataframe_into_mongo(data_emp_df_2013, col_unemployment_2011)
load_dataframe_into_mongo(data_emp_df_2014, col_unemployment_2011)

/opt/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


In [109]:
population_estimate_2011 ='data/population_race_2011.csv'
data_pop_df_2011 = load(population_estimate_2011)
population_estimate_2012 ='data/population_race_2012.csv'
data_pop_df_2012 = load(population_estimate_2012)
population_estimate_2013 ='data/population_race_2013.csv'
data_pop_df_2013 = load(population_estimate_2013)
population_estimate_2014 ='data/population_race_2014.csv'
data_pop_df_2014 = load(population_estimate_2014)

In [119]:
data_pop_df_2014[:2]

,COUNTY,YEAR,TOTAL,WHITE,AFRICANAMERICAN,AMERICAN,ASIAN,NATIVE,MIXED
0,"Alameda County, California",2014,"1,556,249","813,893","197,402","17,780","432,481","15,172","79,521"
1,"Alpine County, California",2014,"1,127",817,5,264,9,0,32


In [112]:
col_population_2011 = bayes_db['population_2011']
load_dataframe_into_mongo(data_pop_df_2011, col_population_2011)
load_dataframe_into_mongo(data_pop_df_2012, col_population_2011)
load_dataframe_into_mongo(data_pop_df_2013, col_population_2011)
load_dataframe_into_mongo(data_pop_df_2014, col_population_2011)

/opt/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


In [113]:
data_pov_income_df_2011 = pd.concat([data_pov_df_2011, data_pov_df_2012, data_pov_df_2013, data_pov_df_2014])
data_unemployment_df_2011 = pd.concat([data_emp_df_2011, data_emp_df_2012, data_emp_df_2013, data_emp_df_2014])
data_pop_race_df_2011 = pd.concat([data_pop_df_2011, data_pop_df_2012, data_pop_df_2013, data_pop_df_2014])

In [114]:
summarize_completeness_uniqueness(data_pop_race_df_2011)

*** How complete is each feature? How many different values does it have? ***


,Column Name,Complete (%),Unique Values
0,COUNTY,0%,58
1,YEAR,0%,3
2,TOTAL,0%,174
3,WHITE,0%,174
4,AFRICANAMERICAN,0%,167
5,AMERICAN,0%,170
6,ASIAN,0%,170
7,NATIVE,0%,162
8,MIXED,0%,171


In [47]:
summarize_completeness_uniqueness(data_pov_income_df_2011)

*** How complete is each feature? How many different values does it have? ***


,Column Name,Complete (%),Unique Values
0,STATE,27%,52
1,CNTYCD,27%,325
2,STATEDESC,27%,52
3,COUNTY,27%,1929
4,YEAR,27%,4
5,POVERTY,27%,8856
6,POVERTYRATE,27%,461
7,INCOME,27%,10663


In [70]:
# cols_to_use = data_unemployment_df_2011.columns.difference(data_pov_income_df_2011.columns)
cols_to_use = data_unemployment_df_2011.columns - data_pov_income_df_2011.columns

/opt/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: using '-' to provide set differences with Indexes is deprecated, use .difference()
  from ipykernel import kernelapp as app


In [76]:
data_unemployment_df_new_2011 = data_unemployment_df_2011[['YEAR', 'STATE', 'CNTYCD', 'FORCE', 'EMPLOYED', 'UNEMPLOYED', 'UNEMPLOYMENTRATE']]

In [133]:
data_demographics_df_2011 = pd.merge(data_pov_income_df_2011, data_unemployment_df_new_2011, 
                                     how='inner', on=['YEAR','STATE','CNTYCD'], copy=False)

In [134]:
data_demographics_race_df_2011 = pd.merge(data_demographics_df_2011, data_pop_race_df_2011,
                                         how='outer', on=['YEAR', 'COUNTY'])

In [135]:
data_demographics_race_df_2011[:50]

,STATE,CNTYCD,STATEDESC,COUNTY,YEAR,POVERTY,POVERTYRATE,INCOME,FORCE,EMPLOYED,UNEMPLOYED,UNEMPLOYMENTRATE,TOTAL,WHITE,AFRICANAMERICAN,AMERICAN,ASIAN,NATIVE,MIXED
0,1,1,AL,Autauga County,2011,"8,152",14.9,"48,863",25836,23677,2159,8.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,3,AL,Baldwin County,2011,"24,728",13.4,"50,144",85045,77418,7627,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13,9,GA,Baldwin County,2011,"11,826",29.2,"34,304",18617,16054,2563,13.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,5,AL,Barbour County,2011,"7,051",29.5,"30,117",9849,8712,1137,11.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,54,1,WV,Barbour County,2011,"3,650",22.8,"33,004",6777,6142,635,9.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1,7,AL,Bibb County,2011,"4,562",22.2,"37,347",8933,7996,937,10.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,13,21,GA,Bibb County,2011,"38,253",25.4,"35,125",71223,63035,8188,11.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1,9,AL,Blount County,2011,"8,505",14.9,"41,940",25123,22939,2184,8.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,47,9,TN,Blount County,2011,"17,800",14.6,"45,539",61745,56702,5043,8.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1,11,AL,Bullock County,2011,"2,909",32.8,"26,038",4833,4272,561,11.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [130]:
data_pop_race_df_2011[data_pop_race_df_2011['COUNTY'] == 'Autauga County']

,COUNTY,YEAR,TOTAL,WHITE,AFRICANAMERICAN,AMERICAN,ASIAN,NATIVE,MIXED


In [136]:
data_demographics_race_df_2011 = data_demographics_race_df_2011[data_demographics_race_df_2011['CNTYCD'] != 0]

In [137]:
summarize_completeness_uniqueness(data_demographics_race_df_2011)

*** How complete is each feature? How many different values does it have? ***


,Column Name,Complete (%),Unique Values
0,STATE,26%,51
1,CNTYCD,26%,322
2,STATEDESC,26%,51
3,COUNTY,27%,1933
4,YEAR,27%,4
5,POVERTY,26%,8648
6,POVERTYRATE,26%,458
7,INCOME,26%,10508
8,FORCE,26%,10791
9,EMPLOYED,26%,10565


In [87]:
summarize_completeness_uniqueness(data_demographics_df_2011)

*** How complete is each feature? How many different values does it have? ***


,Column Name,Complete (%),Unique Values
0,STATE,26%,51
1,CNTYCD,26%,322
2,STATEDESC,26%,51
3,COUNTY,26%,1875
4,YEAR,26%,4
5,POVERTY,26%,8648
6,POVERTYRATE,26%,458
7,INCOME,26%,10508
8,FORCE,26%,10791
9,EMPLOYED,26%,10565


In [88]:
summarize_completeness_uniqueness(data_df_2011)

*** How complete is each feature? How many different values does it have? ***


,Column Name,Complete (%),Unique Values
0,YEAR,100%,5
1,MONTH,100%,12
2,DAY,100%,31
3,TIME24HR,100%,24
4,TIMEMIN,100%,60
5,SUICIDE_ATTEMPTED,100%,2
6,RAILROAD,100%,486
7,INCIDENT_NUM,100%,42981
8,TYPPERS,100%,10
9,JOBCODE,48%,115


In [138]:
consolidated_data_df_2011 = pd.merge(data_df_2011, data_demographics_race_df_2011, how='outer',
                                    on=['YEAR', 'CNTYCD', 'STATE'])

In [139]:
summarize_completeness_uniqueness(consolidated_data_df_2011)

*** How complete is each feature? How many different values does it have? ***


,Column Name,Complete (%),Unique Values
0,YEAR,115%,5
1,MONTH,100%,12
2,DAY,100%,31
3,TIME24HR,100%,24
4,TIMEMIN,100%,60
5,SUICIDE_ATTEMPTED,100%,2
6,RAILROAD,100%,486
7,INCIDENT_NUM,100%,42981
8,TYPPERS,100%,10
9,JOBCODE,48%,115


In [140]:
file_name = 'data/rail_demo_race_casaulties_2011.csv'
consolidated_data_df_2011.to_csv(file_name, sep=',', encoding='utf-8')

In [104]:
consolidated_data_df_2011.shape

(55137, 46)

In [141]:
consolidated_data_df_2011.shape

(55369, 54)

In [103]:
print consolidated_data_df_2011.columns.values

['YEAR' 'MONTH' 'DAY' 'TIME24HR' 'TIMEMIN' 'SUICIDE_ATTEMPTED' 'RAILROAD'
 'INCIDENT_NUM' 'TYPPERS' 'JOBCODE' 'INJURY_NATURE' 'LOCATION' 'IFATAL'
 'AGE' 'DAYSABS' 'DAYSRES' 'STATE' 'TYPRR' 'REGION' 'FATAL' 'CNTYCD'
 'STCNTY' 'ALCOHOL' 'DRUG' 'PHYACT' 'LOCA' 'LOCB' 'LOCC' 'EVENT' 'TOOLS'
 'INJCAUS' 'HZMEXPOS' 'TERMINAT' 'COVERDATA' 'LATITUDE' 'LONGITUDE'
 'NARRATIVE' 'STATEDESC' 'POVERTY' 'POVERTYRATE' 'INCOME' 'FORCE'
 'EMPLOYED' 'UNEMPLOYED' 'UNEMPLOYMENTRATE' 'COUNTY']


In [102]:
consolidated_data_df_2011['COUNTY'] = consolidated_data_df_2011['COUNTY_x']
consolidated_data_df_2011 = consolidated_data_df_2011.drop('COUNTY_y', axis=1)
consolidated_data_df_2011 = consolidated_data_df_2011.drop('COUNTY_x', axis=1)